## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
fabric_rc_location=os.environ['HOME']+"/work/fabric_config/fabric_rc"
if os.path.exists(fabric_rc_location):
    with open(fabric_rc_location, 'r') as f:
        for line in f:
            if line.startswith('export'):
                os.environ[line.split('=')[0].split('export')[1].strip()] = line.split('=')[1].strip()

In [ ]:
ssh_key_file_priv=os.environ["FABRIC_SLICE_PRIVATE_KEY_FILE"]
ssh_key_file_pub=os.environ["FABRIC_SLICE_PUBLIC_KEY_FILE"]

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
slice_name="fabnet"

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager()

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()

## Create Slice

## Site1
- Grab the subnet from NetworkService Sliver for Site 1
- Add route to the Site 2 Subnet
  - IPv4
  ```
  ip route add 10.129.1.0/24 via 10.130.1.1
  ```
  - IPv6
  ```
  ip -6 route add 2602:fcfb:8:1::/64 via 2602:fcfb:4:1::1
  ```
## Site2
- Grab the subnet from NetworkService Sliver for Site 2
- Add route to the Site 2 Subnet
  - IPv4
  ```
  ip route add 10.130.1.0/24 via 10.129.1.1
  ```
  - IPv6
  ```
  ip -6 route add 2602:fcfb:4:1::/64 via 2602:fcfb:8:1::1
  ```
  

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
from fim.slivers.capacities_labels import Flags

# Create topology
t = ExperimentTopology()

# Set capacities
cap = Capacities(core=2, ram=6, disk=10)

site1 = "RENC"
num_nodes = 2
name="node"
i=0
ifcs = []
for x in range(num_nodes):
    # Add node
    nm = f"{name}-{i}"
    
    n = t.add_node(name=f"{name}-{i}", site=site1)

    # Set properties
    n.set_properties(capacities=cap, image_type='qcow2', image_ref='default_rocky_8')
    n.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name=f"{name}-{i}-nic1")
    # Auto Configure: Setup IP Addresses
    n.interface_list[0].flags = Flags(auto_config=True)
    ifcs.append(n.interface_list[0])
        
    i += 1

t.add_network_service(name=f"fabnet-{site1}", nstype=ServiceType.FABNetv6, interfaces=ifcs)

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)


print("Response Status {}".format(status))
if status == Status.OK:
    print("Reservations created {}".format(reservations))
else:
    print(f"Failure: {reservations}")

## Query Slices

In [ ]:
status, slices = slice_manager.slices(excludes=[SliceState.Closing, SliceState.Dead])
#status, slices = slice_manager.slices()

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slices {}".format(slices))
    if len(slices) > 0:
        slice_object=list(filter(lambda s: s.slice_name == slice_name, slices))[0]
else:
    print(f"Failure: {slices}")

## Query Slivers

In [ ]:
status, slivers = slice_manager.slivers(slice_object=slice_object)

print("Response Status {}".format(status))
if status == Status.OK:
    for s in slivers:
        print("=====================")
        print("ReservationId {}".format(s.reservation_id))
        print("State         {}".format(s.reservation_state))
        print("Pending       {}".format(s.pending_state))
        print("Join State    {}".format(s.join_state))
        print("Sliver Type   {}".format(s.sliver_type))
        print("Sliver        {}".format(s.sliver))
        print("Site        {}".format(s.sliver.site))
        if s.sliver_type == "NodeSliver":
            print("Sliver        {}".format(s.sliver.management_ip))
        else:
            print("Sliver        {}".format(s.sliver.gateway.subnet))
        
        if s.sliver_type == "NodeSliver":
            for d in s.sliver.attached_components_info.devices.values():
                print("-------------------------")
                print("COMPONENT        {}".format(d))
                print("-------------------------")
        else:
            for d in s.sliver.interface_info.interfaces.values():
                print("-------------------------")
                print("INTERFACE        {}".format(d))
                print("INTERFACE        {}".format(d.flags))
                print("-------------------------")
                
        print("Notices       {}".format(s.notices))
        
        print("=====================")
else:
    print(f"Failure: {slivers}")

## Delete Slice

In [ ]:
status, result = slice_manager.delete(slice_object=slice_object)

print("Response Status {}".format(status))
print("Response received {}".format(result))